# AI Wine Sommelier RAG

## Wine Review Indexing

https://www.kaggle.com/datasets/christopheiv/winemagdata130k

In [1]:
%pip install -Uq langchain langchain-openai langchain-pinecone langchain-community

Note: you may need to restart the kernel to use updated packages.


In [2]:
!python.exe -m pip install --upgrade pip

In [3]:
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                                                         # 현재 위치의 .env를 읽어와 환경변수로 등록
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")                # .env의 openai_key 값을 OPENAI_API_KEY로 등록
os.environ["LANGSMITH_TRACING"] = 'true'                              # LangSmith 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'  # LangSmith API 엔드포인트 설정
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'                   # LangSmith 프로젝트명 설정
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")          # .env의 langsmith_key 값을 LANGSMITH_API_KEY로 등록
os.environ["TAVILY_API_KEY"] = os.getenv("tavily_key")                 
os.environ["PINECONE_API_KEY"] = os.getenv("pinecone_key")                 


## Pinecone 테스트

In [4]:
# 데이터로드
from langchain_core.documents import Document

documents = [
    Document(page_content="LangChain은 LLM 기반 애플리케이션을 쉽게 만들 수 있는 프레임워크입니다.", metadata={"source": "https://langchain.com/docs", "author": "alice", "page": 1}),
    Document(page_content="ChromaDB는 오픈소스 벡터 데이터베이스입니다.", metadata={"source": "https://chromadb.org/intro", "license": "MIT", "date": "2024-07-01"}),
    Document(page_content="파이썬으로 AI 서비스를 개발할 수 있습니다.", metadata={"source": "https://pythonai.co.kr", "editor": "kim", "page": 7}),
    Document(page_content="LLM은 자연어 처리를 위한 대형 언어 모델을 의미합니다.", metadata={"source": "https://llmwiki.com/info", "author": "bob", "version": "v1.1"}),
    Document(page_content="RAG는 검색과 생성의 결합 방식을 제공합니다.", metadata={"source": "https://rag-search.io", "reviewer": "lee", "section": "summary"}),
    Document(page_content="벡터 데이터베이스는 임베딩된 데이터를 효율적으로 검색할 수 있습니다.", metadata={"source": "https://vectorbase.net", "author": "jin", "topic": "vector"}),
    Document(page_content="LangChain을 이용하면 다양한 AI 파이프라인을 구축할 수 있습니다.", metadata={"source": "https://langchain.com/blog", "editor": "sarah", "date": "2024-06-30"}),
    Document(page_content="OpenAI의 GPT 모델은 텍스트 생성에 특화되어 있습니다.", metadata={"source": "https://openai.com/gpt", "lang": "ko", "page": 5}),
    Document(page_content="파이썬은 AI 및 데이터 분석 분야에서 널리 사용되는 언어입니다.", metadata={"source": "https://python.org/usecases", "author": "chun", "updated": "2024-05"}),
    Document(page_content="Streamlit은 파이썬으로 대시보드를 쉽게 만들 수 있는 프레임워크입니다.", metadata={"source": "https://streamlit.io/start", "editor": "park", "date": "2024-04-28"}),
    Document(page_content="Dense Retrieval은 임베딩 벡터를 이용한 검색 방식을 의미합니다.", metadata={"source": "https://retrieval.ai/dense", "type": "tech", "page": 3}),
    Document(page_content="Pandas 라이브러리는 데이터 분석에 자주 사용됩니다.", metadata={"source": "https://pandas.pydata.org/about", "maintainer": "koh", "section": "intro"}),
    Document(page_content="메타데이터 필터링은 검색 결과의 품질을 높여줍니다.", metadata={"source": "https://search.com/metadata", "author": "seo", "feature": "filter"}),
    Document(page_content="SelfQueryRetriever는 자연어 쿼리를 임베딩 쿼리로 변환해줍니다.", metadata={"source": "https://selfquery.ai", "editor": "min", "date": "2024-05-12"}),
    Document(page_content="프롬프트 엔지니어링은 LLM의 성능을 극대화하는 방법입니다.", metadata={"source": "https://prompting.dev/guide", "author": "yang", "topic": "prompt"}),
    Document(page_content="HyDE 기법은 하이브리드 검색에 사용됩니다.", metadata={"source": "https://hyde-tech.com", "reviewer": "kang", "version": "2024.1"}),
    Document(page_content="CoT는 복잡한 문제를 단계적으로 해결하는 프롬프트 기법입니다.", metadata={"source": "https://cotprompt.org", "editor": "jung", "date": "2023-12-01"}),
    Document(page_content="문서 임베딩은 텍스트를 고차원 벡터로 변환하는 과정입니다.", metadata={"source": "https://embedding.ai/intro", "section": "embedding", "author": "song"}),
    Document(page_content="CrewAI는 멀티 에이전트 시스템 구현을 돕는 툴입니다.", metadata={"source": "https://crew.ai/docs", "lang": "ko", "page": 9}),
    Document(page_content="Fine-tuning은 사전학습 모델을 특정 도메인에 맞게 재학습시키는 과정입니다.", metadata={"source": "https://finetune.ai/guide", "editor": "jeon", "date": "2024-01-30"})
]

In [5]:
from langchain_openai import OpenAIEmbeddings           # 문서 텍스트를 임베딩 벡터로 변환
from langchain_pinecone import PineconeVectorStore      # Pinecone 인덱스에 임베딩/문서를 저장하는 벡터스토어

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')


# 문서 -> 임베딩 -> Pinecone 업로드
vector_store = PineconeVectorStore.from_documents(
    documents,                      # 업로드할 Document(청크) 리스트
    embeddings,                     # 임베딩 모델
    index_name = 'pinecone-first'   # Pinecone 인덱스 이름
      
)

c:\nlp\nlp_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# 질의를 임베딩 한 뒤 가장 유사한 Document 리스트 반환
retrievals = vector_store.similarity_search('벡터 데이터베이스란?')
retrievals

[Document(id='b1990659-cd96-443a-adf3-42f1a98bc25e', metadata={'author': 'jin', 'source': 'https://vectorbase.net', 'topic': 'vector'}, page_content='벡터 데이터베이스는 임베딩된 데이터를 효율적으로 검색할 수 있습니다.'),
 Document(id='e6b3f38f-3091-45b4-a883-5633255aef66', metadata={'date': '2024-07-01', 'license': 'MIT', 'source': 'https://chromadb.org/intro'}, page_content='ChromaDB는 오픈소스 벡터 데이터베이스입니다.'),
 Document(id='1195abb8-b253-4e1f-8628-aeaa876ace80', metadata={'page': 3.0, 'source': 'https://retrieval.ai/dense', 'type': 'tech'}, page_content='Dense Retrieval은 임베딩 벡터를 이용한 검색 방식을 의미합니다.'),
 Document(id='8042bc2c-44e2-44b5-8597-c5e95766a71e', metadata={'author': 'song', 'section': 'embedding', 'source': 'https://embedding.ai/intro'}, page_content='문서 임베딩은 텍스트를 고차원 벡터로 변환하는 과정입니다.')]

In [7]:
retrievals[1].page_content

'ChromaDB는 오픈소스 벡터 데이터베이스입니다.'

In [8]:
# PineconeVectorStore를 Retriever로 변환해 Top-k 검색
retriever = vector_store.as_retriever(
    search_type = 'similarity',
    search_kwargs =  {'k' : 3}
)

retriever.invoke('벡터 데이터베이스란?')

[Document(id='b1990659-cd96-443a-adf3-42f1a98bc25e', metadata={'author': 'jin', 'source': 'https://vectorbase.net', 'topic': 'vector'}, page_content='벡터 데이터베이스는 임베딩된 데이터를 효율적으로 검색할 수 있습니다.'),
 Document(id='ca4d3883-883e-4b66-9051-4f8213a556ba', metadata={'author': 'jin', 'source': 'https://vectorbase.net', 'topic': 'vector'}, page_content='벡터 데이터베이스는 임베딩된 데이터를 효율적으로 검색할 수 있습니다.'),
 Document(id='e6b3f38f-3091-45b4-a883-5633255aef66', metadata={'date': '2024-07-01', 'license': 'MIT', 'source': 'https://chromadb.org/intro'}, page_content='ChromaDB는 오픈소스 벡터 데이터베이스입니다.')]

In [9]:
from langchain_community.document_loaders import CSVLoader          # CSV의 각 행(row)를 하나의 Document 객체로 변환하는 로더

loader = CSVLoader('winemag-data-130k-v2.csv', encoding='utf-8')    # CSV 경로/인코딩 지정
docs = loader.load()                                                # CSV를 Document 리스트로 로드(행 단우ㅏ)
print(len(docs))


129971


In [10]:
docs = docs[:30000]
print(len(docs))

30000


In [11]:
# CSV 로더 결과 확인
for i, doc in enumerate(docs[:2], 1):   # Document 중 앞 2개 확인(index 1부터 시작)
    print(f"{i} : {type(doc)}")         
    print(f"{doc.metadata}")
    print(f"{doc.page_content}")
    print()

1 : <class 'langchain_core.documents.base.Document'>
{'source': 'winemag-data-130k-v2.csv', 'row': 0}
: 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia

2 : <class 'langchain_core.documents.base.Document'>
{'source': 'winemag-data-130k-v2.csv', 'row': 1}
: 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
tast

In [14]:
vector_store = PineconeVectorStore(
    index_name= "winemag-data-130k-v2", # 업로드 대상 인덱스 이름
    embedding = embeddings              # 임베딩 모델
)

batch_size = 300                             # 한 번에 업로드할 Document 개수

for i in range(0, len(docs), batch_size):       # 전체 docs를 batch_size 단위로 순회
    batch_data = docs[i: i + batch_size]
    vector_store.add_documents(batch_data)      # 배치 Document들을 임베딩 후 Pinecone 업로드
    print(f"index: {i} ~ {i+batch_size}")
    

index: 0 ~ 300
index: 300 ~ 600
index: 600 ~ 900
index: 900 ~ 1200
index: 1200 ~ 1500
index: 1500 ~ 1800
index: 1800 ~ 2100
index: 2100 ~ 2400
index: 2400 ~ 2700
index: 2700 ~ 3000
index: 3000 ~ 3300
index: 3300 ~ 3600
index: 3600 ~ 3900
index: 3900 ~ 4200
index: 4200 ~ 4500
index: 4500 ~ 4800
index: 4800 ~ 5100
index: 5100 ~ 5400
index: 5400 ~ 5700
index: 5700 ~ 6000
index: 6000 ~ 6300
index: 6300 ~ 6600
index: 6600 ~ 6900
index: 6900 ~ 7200
index: 7200 ~ 7500
index: 7500 ~ 7800
index: 7800 ~ 8100
index: 8100 ~ 8400
index: 8400 ~ 8700
index: 8700 ~ 9000
index: 9000 ~ 9300
index: 9300 ~ 9600
index: 9600 ~ 9900
index: 9900 ~ 10200
index: 10200 ~ 10500
index: 10500 ~ 10800
index: 10800 ~ 11100
index: 11100 ~ 11400
index: 11400 ~ 11700
index: 11700 ~ 12000
index: 12000 ~ 12300
index: 12300 ~ 12600
index: 12600 ~ 12900
index: 12900 ~ 13200
index: 13200 ~ 13500
index: 13500 ~ 13800
index: 13800 ~ 14100
index: 14100 ~ 14400
index: 14400 ~ 14700
index: 14700 ~ 15000
index: 15000 ~ 15300
index

## Retrieval & Generation
1. 텍스트/이미지 입력으로 요리에 설명 chain
2. 요리설명텍스트 벡터db조회 chain
3. 요리설명/리뷰검색을 가지고 와인추천 응답 chain


### 요리설명 chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate  # 프롬프트 템플릿 관련 클래스
from langchain.chat_models import init_chat_model  # OpenAI 채팅 모델을 LangChain에서 쓰기 위한 함수
from langchain_core.output_parsers import StrOutputParser  # LLM 출력에서 문자열만 추출하는 파서
from langchain_core.runnables import RunnableLambda  # 일반 함수를 Runnable로 감싸주는 도구

def describe_dish_flavor(query : dict):  # 사용자 입력(query)을 받아 체인을 생성하는 함수
    prompt = ChatPromptTemplate.from_messages([  # system / human 메시지로 프롬프트 구성
        ('system', ''' ... '''),  # LLM의 페르소나, 작성 규칙, 예시 등을 정의하는 시스템 메시지
        ('human', '사용자가 제공한 이미지의 요리명과 풍미를 잘 묘사해주세요.')  # 기본 사용자 요청 메시지
    ])    
    
    temp = []  # 이미지와 텍스트를 담기 위한 임시 리스트

    if query.get('image_urls'):  # query의 이미지 URL 키가 있는 경우
        temp += [{"image_url" : image_url} for image_url in query.get('image_urls')]  # temp에 이미지 정보 추가

    if query.get('text'):  # 텍스트 입력이 있다면
        temp += [{"text" : query.get('text')}]  # temp에 텍스트 정보 추가
        
    #위에서 만든 멀티모달 블록(temp)를 human 메시지로 프롬프트에 추가
    prompt += HumanMessagePromptTemplate.from_template(temp)  
    
        
    llm = init_chat_model('openai:gpt-4.1-mini')  # GPT-4.1-mini 모델 초기화
    output_parser = StrOutputParser()  # 문자열 출력 파서 생성
    
    chain = prompt | llm | output_parser  # 프롬프트 → LLM → 파서 순서로 체인 구성
    
    return chain 

dish_flavor_chain = RunnableLambda(describe_dish_flavor)  # 입력을 받아 체인을 만들어 실행하는 Runnable

response = dish_flavor_chain.invoke({  # Runnable 실행 (.invoke 필수)
    'text' : '',
    'image_urls' : [
        "https://search.pstatic.net/common/?src=...",
        "https://search.pstatic.net/common/?src=...",
        "https://search.pstatic.net/common/?src=..."
    ]  # 요리 사진 URL 여러장
})

print(response)  # 결과 출력


1. 첫 번째 요리인 족발은 은은한 간장과 향신료가 스며든 쫄깃한 육질이 혀끝에서 부드럽게 녹으며 고소하면서도 감칠맛이 깊게 전해집니다.

2. 두 번째 요리인 닭발구이는 매콤한 고추장 소스가 강렬하게 입안을 자극하며, 쫄깃한 식감과 함께 불맛이 어우러져 중독성 있는 풍미를 선사합니다.

3. 세 번째 요리인 제육볶음은 고추장의 매콤함과 달큰한 양념이 조화를 이루고, 얇게 썬 돼지고기가 부드럽게 씹히며 달큰하고 짭짤한 맛이 입안 가득 퍼집니다.
